# 用pytorch搭建全连接神经网络，对Fashion-MNIST数据集中的时尚物品进行分类

# 1 下载Fasion-MNIST数据集

`Fashion-MNIST`是一个替代原始的[MNIST手写数字数据集](http://yann.lecun.com/exdb/mnist/)的`另一个`图像数据集。 它是由Zalando（一家德国的时尚科技公司）旗下的[研究部门](https://research.zalando.com/)提供。其涵盖了来自10种类别的共7万个不同商品的正面图片。Fashion-MNIST的大小、格式和训练集/测试集划分与原始的MNIST完全一致。60000/10000的训练测试数据划分，28x28的灰度图片。你可以直接用它来测试你的机器学习和深度学习算法性能，且**不需要**改动任何的代码。

Fashion-MNIST 数据集的[中文文档说明](https://github.com/zalandoresearch/fashion-mnist/blob/master/README.zh-CN.md)


In [2]:
import torch  # 导入pytorch
from torch import nn, optim  # 导入神经网络与优化器对应的类
import torch.nn.functional as F 
from torchvision import datasets, transforms ## 导入数据集与数据预处理的方法

In [3]:
# 数据预处理：标准化图像数据，使得灰度数据在-1到+1之间
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,))])

# 下载Fashion-MNIST训练集数据，并构建训练集数据载入器trainloader,每次从训练集中载入64张图片，每次载入都打乱顺序
trainset = datasets.FashionMNIST('data/', download=False, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# 下载Fashion-MNIST测试集数据，并构建测试集数据载入器trainloader,每次从测试集中载入64张图片，每次载入都打乱顺序
testset = datasets.FashionMNIST('data/', download=False, train=False, transform=transform) #download=True 时下载Fashion-MNIST数据，若已下载设置为False
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

D:\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


## 显示图片

In [4]:
image, label = next(iter(trainloader))

# image图片中有64张图片，我们查看索引为2的图片
imagedemo = image[3]
imagedemolabel = label[3]

imagedemo = imagedemo.reshape((28,28))

import matplotlib.pyplot as plt
%matplotlib inline
#plt.imshow(imagedemo)

labellist = ['T恤','裤子','套衫','裙子','外套','凉鞋','汗衫','运动鞋','包包','靴子']
print(f'这张图片对应的标签是 {labellist[imagedemolabel]}')

这张图片对应的标签是 外套


# 2 打开数据集中的图片

In [5]:
image, label = next(iter(trainloader))

In [6]:
image.shape

torch.Size([64, 1, 28, 28])

image包含了64张28 * 28的灰度图片，1代表单通道，也就是灰度

In [7]:
label

tensor([0, 8, 4, 9, 3, 8, 4, 5, 8, 1, 5, 0, 6, 9, 9, 8, 7, 3, 6, 9, 8, 8, 8, 6,
        6, 4, 7, 5, 2, 8, 5, 6, 2, 1, 6, 5, 6, 7, 8, 8, 0, 2, 4, 5, 2, 0, 9, 6,
        3, 3, 6, 3, 7, 7, 4, 2, 2, 0, 8, 4, 7, 0, 3, 8])

label包含了image里面64张图片对应的标签

| 标注编号 | 描述 |
| --- | --- |
| 0 | T-shirt/top（T恤）|
| 1 | Trouser（裤子）|
| 2 | Pullover（套衫）|
| 3 | Dress（裙子）|
| 4 | Coat（外套）|
| 5 | Sandal（凉鞋）|
| 6 | Shirt（汗衫）|
| 7 | Sneaker（运动鞋）|
| 8 | Bag（包）|
| 9 | Ankle boot（踝靴）|

## 打开训练集中的一张图片看看

In [8]:
imagedemo = image[2]
imagedemolabel = label[2]

In [9]:
imagedemo.shape

torch.Size([1, 28, 28])

In [10]:
imagedemo = imagedemo.reshape((28,28))

In [11]:
imagedemo.shape

torch.Size([28, 28])

In [12]:
imagedemolabel

tensor(4)

In [13]:
import matplotlib.pyplot as plt
%matplotlib inline

In [14]:
#plt.imshow(imagedemo)

labellist = ['T恤','裤子','套衫','裙子','外套','凉鞋','汗衫','运动鞋','包包','靴子']
print(f'这张图片对应的标签是 {labellist[imagedemolabel]}')

这张图片对应的标签是 外套


## 查看一张图片和标签完整的代码

In [15]:
image, label = next(iter(trainloader))

# image图片中有64张图片，我们查看索引为2的图片
imagedemo = image[3]
imagedemolabel = label[3]

imagedemo = imagedemo.reshape((28,28))

import matplotlib.pyplot as plt
%matplotlib inline
#plt.imshow(imagedemo)

labellist = ['T恤','裤子','套衫','裙子','外套','凉鞋','汗衫','运动鞋','包包','靴子']
print(f'这张图片对应的标签是 {labellist[imagedemolabel]}')

这张图片对应的标签是 裙子


# 2 搭建并训练四层全连接神经网络

神经网络的输入为28 * 28 = 784 个像素<br>
第一个隐含层包含256个神经元<br>
第二个隐含层包含128个神经元<br>
第三个隐含层包含64个神经元<br>
输出层输出10个结果，对应图片的10种分类<br>

![全连接神经网络](https://upload-images.jianshu.io/upload_images/13714448-c8b5fa11504798bf.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)

In [16]:
from torch import nn, optim
import torch.nn.functional as F

class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)
        
    def forward(self, x):
        # make sure input tensor is flattened
        x = x.view(x.shape[0], -1)
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.log_softmax(self.fc4(x), dim=1)
        
        return x

In [22]:
# 对上面定义的Classifier类进行实例化
model = Classifier()

# 定义损失函数为负对数损失函数
criterion = nn.NLLLoss()

# 优化方法为Adam梯度下降方法，学习率为0.003
optimizer = optim.Adam(model.parameters(), lr=0.003)

# 对训练集的全部数据学习15遍，这个数字越大，训练时间越长
epochs = 15

# 将每次训练的训练误差和测试误差存储在这两个列表里，后面绘制误差变化折线图用
train_losses, test_losses = [], []

print('开始训练')
for e in range(epochs):
    running_loss = 0
    
    # 对训练集中的所有图片都过一遍
    for images, labels in trainloader:
        # 将优化器中的求导结果都设为0，否则会在每次反向传播之后叠加之前的
        optimizer.zero_grad()
        
        # 对64张图片进行推断，计算损失函数，反向传播优化权重，将损失求和
        log_ps = model(images)
        loss = criterion(log_ps, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    # 每次学完一遍数据集，都进行以下测试操作
    else:
        test_loss = 0
        accuracy = 0
        # 测试的时候不需要开自动求导和反向传播
        with torch.no_grad():
            # 关闭Dropout
            model.eval()
            
            # 对测试集中的所有图片都过一遍
            for images, labels in testloader:
                # 对传入的测试集图片进行正向推断、计算损失，accuracy为测试集一万张图片中模型预测正确率
                log_ps = model(images)
                test_loss += criterion(log_ps, labels)
                ps = torch.exp(log_ps)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                
                # 等号右边为每一批64张测试图片中预测正确的占比
                accuracy += torch.mean(equals.type(torch.FloatTensor))
        # 恢复Dropout
        model.train()
        #保存模型
        model.save("F_cnn.pth")
        # 将训练误差和测试误差存在两个列表里，后面绘制误差变化折线图用
        train_losses.append(running_loss/len(trainloader))
        test_losses.append(test_loss/len(testloader))

        print("训练集学习次数: {}/{}.. ".format(e+1, epochs),
              "训练误差: {:.3f}.. ".format(running_loss/len(trainloader)),
              "测试误差: {:.3f}.. ".format(test_loss/len(testloader)),
              "模型分类准确率: {:.3f}".format(accuracy/len(testloader)))

开始训练
训练集学习次数: 1/15..  训练误差: 0.515..  测试误差: 0.436..  模型分类准确率: 0.842
训练集学习次数: 2/15..  训练误差: 0.390..  测试误差: 0.411..  模型分类准确率: 0.855
训练集学习次数: 3/15..  训练误差: 0.354..  测试误差: 0.418..  模型分类准确率: 0.856
训练集学习次数: 4/15..  训练误差: 0.330..  测试误差: 0.420..  模型分类准确率: 0.848
训练集学习次数: 5/15..  训练误差: 0.318..  测试误差: 0.354..  模型分类准确率: 0.875
训练集学习次数: 6/15..  训练误差: 0.302..  测试误差: 0.358..  模型分类准确率: 0.873
训练集学习次数: 7/15..  训练误差: 0.293..  测试误差: 0.365..  模型分类准确率: 0.874
训练集学习次数: 8/15..  训练误差: 0.282..  测试误差: 0.359..  模型分类准确率: 0.880
训练集学习次数: 9/15..  训练误差: 0.272..  测试误差: 0.380..  模型分类准确率: 0.878
训练集学习次数: 10/15..  训练误差: 0.268..  测试误差: 0.358..  模型分类准确率: 0.879
训练集学习次数: 11/15..  训练误差: 0.263..  测试误差: 0.374..  模型分类准确率: 0.875
训练集学习次数: 12/15..  训练误差: 0.255..  测试误差: 0.373..  模型分类准确率: 0.876
训练集学习次数: 13/15..  训练误差: 0.251..  测试误差: 0.369..  模型分类准确率: 0.876
训练集学习次数: 14/15..  训练误差: 0.239..  测试误差: 0.384..  模型分类准确率: 0.879
训练集学习次数: 15/15..  训练误差: 0.238..  测试误差: 0.365..  模型分类准确率: 0.882


# 3 验证模型效果

## 绘制训练误差和测试误差随学习次数增加的变化

In [17]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.plot(train_losses, label='Training loss')
plt.plot(test_losses, label='Validation loss')
#plt.legend()

可以看到，虽然训练误差一直在下降，但测试误差居高不下，我们的神经网络仿佛一个高分低能的同学，平时把所有课后题答案都死记硬背下来，一到考试见到新题的时候就不会做了。

虽然高分低能，但大部分时候依旧能做出正确判断，但是有时候预测概率只有百分之三四十的把握，不能做到十有八九的确定。

In [18]:
dataiter = iter(testloader)
images, labels = dataiter.next()
img = images[0]
img = img.reshape((28,28)).numpy()
#plt.imshow(img)

# 将测试图片转为一维的列向量
img = torch.from_numpy(img)
img = img.view(1, 784)

# 进行正向推断，预测图片所在的类别
with torch.no_grad():
    output = model.forward(img)
ps = torch.exp(output)

top_p, top_class = ps.topk(1, dim=1)
labellist = ['T恤','裤子','套衫','裙子','外套','凉鞋','汗衫','运动鞋','包包','靴子']
prediction = labellist[top_class]
probability = float(top_p)
print(f'神经网络猜测图片里是 {prediction}，概率为{probability*100}%')

NameError: name 'model' is not defined

# 4 采用Dropout方法防止过拟合

我们可以采用Dropout的方法，也就是在每次正向推断训练神经元的时候随机“掐死”一部分神经元，阻断其输入输出，这样可以起到正则化的作用。

可以理解为，皇上雨露均沾，今天受宠，明天可能就被打入冷宫，这样就防止了杨贵妃那样的“三千宠爱在一身”，从而防止了某些神经元一家独大，成为话题领袖，只手遮天。

所有神经元处于平等地位，防止过拟合。

In [19]:
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)
        
        # 构造Dropout方法，在每次训练过程中都随机“掐死”百分之二十的神经元，防止过拟合。
        self.dropout = nn.Dropout(p=0.2)
        
    def forward(self, x):
        # 确保输入的tensor是展开的单列数据，把每张图片的通道、长度、宽度三个维度都压缩为一列
        x = x.view(x.shape[0], -1)
        
        # 在训练过程中对隐含层神经元的正向推断使用Dropout方法
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))
        
        # 在输出单元不需要使用Dropout方法
        x = F.log_softmax(self.fc4(x), dim=1)
        
        return x

In [37]:
# 对上面定义的Classifier类进行实例化
model = Classifier()

# 定义损失函数为负对数损失函数
criterion = nn.NLLLoss()

# 优化方法为Adam梯度下降方法，学习率为0.003
optimizer = optim.Adam(model.parameters(), lr=0.003)

# 对训练集的全部数据学习15遍，这个数字越大，训练时间越长
epochs = 15

# 将每次训练的训练误差和测试误差存储在这两个列表里，后面绘制误差变化折线图用
train_losses, test_losses = [], []

print('开始训练')
for e in range(epochs):
    running_loss = 0
    
    # 对训练集中的所有图片都过一遍
    for images, labels in trainloader:
        # 将优化器中的求导结果都设为0，否则会在每次反向传播之后叠加之前的
        optimizer.zero_grad()
        
        # 对64张图片进行推断，计算损失函数，反向传播优化权重，将损失求和
        log_ps = model(images)
        loss = criterion(log_ps, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    # 每次学完一遍数据集，都进行以下测试操作
    else:
        test_loss = 0
        accuracy = 0
        # 测试的时候不需要开自动求导和反向传播
        with torch.no_grad():
            # 关闭Dropout
            model.eval()
            
            # 对测试集中的所有图片都过一遍
            for images, labels in testloader:
                # 对传入的测试集图片进行正向推断、计算损失，accuracy为测试集一万张图片中模型预测正确率
                log_ps = model(images)
                test_loss += criterion(log_ps, labels)
                ps = torch.exp(log_ps)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                
                # 等号右边为每一批64张测试图片中预测正确的占比
                accuracy += torch.mean(equals.type(torch.FloatTensor))
        # 恢复Dropout
        model.train()
        #保存模型
        torch.save(model, "F_cnn2.pth")
        # 将训练误差和测试误差存在两个列表里，后面绘制误差变化折线图用
        train_losses.append(running_loss/len(trainloader))
        test_losses.append(test_loss/len(testloader))

        print("训练集学习次数: {}/{}.. ".format(e+1, epochs),
              "训练误差: {:.3f}.. ".format(running_loss/len(trainloader)),
              "测试误差: {:.3f}.. ".format(test_loss/len(testloader)),
              "模型分类准确率: {:.3f}".format(accuracy/len(testloader)))

开始训练
训练集学习次数: 1/15..  训练误差: 0.605..  测试误差: 0.455..  模型分类准确率: 0.836
训练集学习次数: 2/15..  训练误差: 0.480..  测试误差: 0.441..  模型分类准确率: 0.844
训练集学习次数: 3/15..  训练误差: 0.452..  测试误差: 0.426..  模型分类准确率: 0.845
训练集学习次数: 4/15..  训练误差: 0.431..  测试误差: 0.422..  模型分类准确率: 0.850
训练集学习次数: 5/15..  训练误差: 0.417..  测试误差: 0.405..  模型分类准确率: 0.857
训练集学习次数: 6/15..  训练误差: 0.407..  测试误差: 0.391..  模型分类准确率: 0.857
训练集学习次数: 7/15..  训练误差: 0.405..  测试误差: 0.403..  模型分类准确率: 0.858
训练集学习次数: 8/15..  训练误差: 0.394..  测试误差: 0.384..  模型分类准确率: 0.863
训练集学习次数: 9/15..  训练误差: 0.393..  测试误差: 0.391..  模型分类准确率: 0.866
训练集学习次数: 10/15..  训练误差: 0.389..  测试误差: 0.390..  模型分类准确率: 0.865
训练集学习次数: 11/15..  训练误差: 0.375..  测试误差: 0.376..  模型分类准确率: 0.865
训练集学习次数: 12/15..  训练误差: 0.380..  测试误差: 0.386..  模型分类准确率: 0.864
训练集学习次数: 13/15..  训练误差: 0.377..  测试误差: 0.386..  模型分类准确率: 0.866
训练集学习次数: 14/15..  训练误差: 0.373..  测试误差: 0.378..  模型分类准确率: 0.867
训练集学习次数: 15/15..  训练误差: 0.373..  测试误差: 0.377..  模型分类准确率: 0.869


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.plot(train_losses, label='Training loss')
plt.plot(test_losses, label='Validation loss')
#plt.legend()

可以看到，训练误差和测试误差都随学习次数增加逐渐降低，没有出现“高分低能”和“死记硬背”的过拟合现象，这其实是Dropout正则化的功劳。

In [34]:
model.eval()
dataiter = iter(testloader)
images, labels = dataiter.next()
img = images[0]

img = img.reshape((28,28)).numpy()
#plt.imshow(img)

# 将测试图片转为一维的列向量
img = torch.from_numpy(img)
img = img.view(1, 784)

# 进行正向推断，预测图片所在的类别
with torch.no_grad():
    output = model.forward(img)
ps = torch.exp(output)

top_p, top_class = ps.topk(1, dim=1)
labellist = ['T恤','裤子','套衫','裙子','外套','凉鞋','汗衫','运动鞋','包包','靴子']
prediction = labellist[top_class]
probability = float(top_p)
print(f'神经网络猜测图片里是 {prediction}，概率为{probability*100}%')

<class 'torch.Tensor'>
(28, 28)
<class 'torch.Tensor'>
神经网络猜测图片里是 T恤，概率为49.986106157302856%


In [49]:
import cv2
#加载模型
model=torch.load("F_cnn2.pth")
model.eval()
#读取图片为单通道
img = cv2.imread('E:/fashion-mnist/data/fashion/test/18.jpg',0)
img = cv2.resize(img, (28, 28), interpolation=cv2.INTER_CUBIC)

# 将测试图片转为一维的列向量
img = torch.from_numpy(img).float()
img = img.view(1, 784)

# 进行正向推断，预测图片所在的类别
with torch.no_grad():
    output = model.forward(img)
ps = torch.exp(output)

top_p, top_class = ps.topk(1, dim=1)
labellist = ['T恤','裤子','套衫','裙子','外套','凉鞋','汗衫','运动鞋','包包','靴子']
prediction = labellist[top_class]
probability = float(top_p)
if probability < 0.2:
    print("未属于训练集中的商品类别/不支持类别")
else:
    print(f'神经网络猜测图片里是 {prediction}，概率为{probability*100}%')

神经网络猜测图片里是 包包，概率为100.0%
